 # Deconvolution (Transposed Convolution)
 ### DCGAN 的生成器，會使用到逆卷積操作來放大 noise vector (z)，生成影像。
<img src="../notebook_material/dcgan_gen.png" alt="Drawing" align="left"/>

## 如何實現?

<img src="../notebook_material/no_padding_no_strides_transposed.gif" alt="Drawing" align="left" style="width: 150px;"/>
<br>
<b align="center"> 以 No zero padding, unit strides, transposed 做示範</b>

In [12]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import numpy as np
import tensorflow as tf

In [13]:
input_data=np.array([
               [[1,4],
                [7,2]]
            ])
input_data = input_data.reshape(-1, input_data.shape[1], input_data.shape[2], input_data.shape[0])


filter_weights=np.array([ 
              [[[ 1, 0, 1],
                [-1, 1, 0],
                [ 0,-1, 0]]] 
           ])
filter_weights = filter_weights.reshape(filter_weights.shape[2], filter_weights.shape[3], filter_weights.shape[1], -1)

# input shape=[batch_size, height, width, channel]
print('input_data shape: ', input_data.shape)

# filter shape=[filter_height, filter_width, in_channel, out_channel]
print('filter_weights shape:', filter_weights.shape)

input_data shape:  (1, 2, 2, 1)
filter_weights shape: (3, 3, 1, 1)


### 1. 手刻

In [14]:
# unit stride
def conv(input, Filter):
    input = tf.constant(input, tf.float32)
    Filter = tf.constant(Filter, tf.float32)
    return tf.nn.conv2d(input, Filter, strides=(1,1,1,1), padding='VALID')

# adding 2x2 zeros border around input
def zero_padding(input):
    # 請輸入程式碼
    batch_size, height, width, channel = input.shape
    output = np.zeros([batch_size, height + 4, width + 4, channel],np.float32)
    
    for i in range(batch_size):
        for j in range(channel):
            output[i, 2:2+height, 2:2+width, j] = input[i, :height ,:width, j]
    return output

In [15]:
#handcraft transposed convolution
init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)

padding_input = zero_padding(input_data)
deconv_hand = sess.run(conv(padding_input, filter_weights))


In [16]:
print('Shape before deconvolution:', input_data.shape)
print('Shape after deconvolution: ', deconv_hand.shape)

Shape before deconvolution: (1, 2, 2, 1)
Shape after deconvolution:  (1, 4, 4, 1)


### 2. Tensorflow 內建 conv2d_transpose 

In [17]:
def tf_conv2d_transpose(input,Filter):
    #input_shape=[batch_size,height,width,channel]
    input = tf.constant(input, tf.float32)
    input_shape = input.get_shape().as_list()
    
    #filter_shape=[height,width,out_c,in_c]
    Filter = np.swapaxes(Filter, 2,3)
    Filter = tf.constant(Filter, tf.float32)
    filter_shape=Filter.get_shape().as_list() 
    
    output_shape=[input_shape[0], input_shape[1]*2 , input_shape[2]*2 , filter_shape[2]]

    deconv=tf.nn.conv2d_transpose(input,Filter,output_shape=output_shape,
        strides=[1, 1, 1, 1], padding='VALID')
    return deconv


In [18]:
#tesndorflow transposed convolution layer
tf.reset_default_graph()
init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)

filter_weights = np.rot90(filter_weights, k=2, axes=(0,1)) # tf_conv2d_transpose 中， filter 會被逆旋轉 180 度
                                                           # 所以先對 height, width 方向順旋轉 180 度
                                                           # 出來的結果才會跟前面一樣
deconv_tf = sess.run(tf_conv2d_transpose(input_data, filter_weights)) 
print('Shape before deconvolution:', input_data.shape)
print('Shape after deconvolution: ', deconv_tf.shape)

Shape before deconvolution: (1, 2, 2, 1)
Shape after deconvolution:  (1, 4, 4, 1)


### 兩種方法結果是否相同

In [19]:
print(np.equal(deconv_tf, deconv_hand).all())

True
